## Product Recommendation

Recommend products to customers by analysing what they might like based on rating by similar customers.


In [1]:
import numpy as np
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt

### Merge Data

In [2]:
productData = pd.read_csv('./data/brazilian-ecommerce/olist_products_dataset.csv')
customerData = pd.read_csv('./data/brazilian-ecommerce/olist_customers_dataset.csv')
orderItData = pd.read_csv('./data/brazilian-ecommerce/olist_order_items_dataset.csv')
orderData = pd.read_csv('./data/brazilian-ecommerce/olist_orders_dataset.csv')
categoryData = pd.read_csv('./data/brazilian-ecommerce/product_category_name_translation.csv')
reviewData = pd.read_csv('./data/brazilian-ecommerce/olist_order_reviews_dataset.csv')


In [3]:
joinData = customerData.merge(orderData, on='customer_id', how='left')
joinData = joinData.merge(orderItData, on='order_id', how='left')
joinData = joinData.merge(productData, on='product_id', how='left')
joinData = joinData.merge(categoryData, on = 'product_category_name')
joinData = joinData = joinData.merge(reviewData, on='order_id', how='left')
joinData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111686 entries, 0 to 111685
Data columns (total 33 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_id                    111686 non-null  object 
 1   customer_unique_id             111686 non-null  object 
 2   customer_zip_code_prefix       111686 non-null  int64  
 3   customer_city                  111686 non-null  object 
 4   customer_state                 111686 non-null  object 
 5   order_id                       111686 non-null  object 
 6   order_status                   111686 non-null  object 
 7   order_purchase_timestamp       111686 non-null  object 
 8   order_approved_at              111672 non-null  object 
 9   order_delivered_carrier_date   110523 non-null  object 
 10  order_delivered_customer_date  109280 non-null  object 
 11  order_estimated_delivery_date  111686 non-null  object 
 12  order_item_id                 

## Recommendation by Popularity


#### Data Cleaning

In [4]:
Data = pd.DataFrame(joinData[['customer_state', 'product_category_name_english', 'product_id']])
Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111686 entries, 0 to 111685
Data columns (total 3 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   customer_state                 111686 non-null  object
 1   product_category_name_english  111686 non-null  object
 2   product_id                     111686 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


#### Customer State

In [5]:
stateCount = pd.DataFrame(Data['customer_state'].value_counts())
stateCount = stateCount.reset_index()
stateCount = stateCount.rename(columns = {'index': 'customer_state', 'customer_state':'customer_state_count'})
stateCount

customer_state  customer_state_count
0              SP                 47052
1              RJ                 14440
2              MG                 13024
3              RS                  6186
4              PR                  5685
5              SC                  4128
6              BA                  3758
7              DF                  2400
8              GO                  2298
9              ES                  2251
10             PE                  1796
11             CE                  1465
12             PA                  1068
13             MT                  1046
14             MS                   822
15             MA                   818
16             PB                   593
17             PI                   536
18             RN                   525
19             AL                   444
20             SE                   383
21             TO                   310
22             RO                   272
23             AM                   164
24             AC                    90
25             AP                    82
26             RR                    50

In [6]:
sb.catplot(y = 'customer_state', data = Data, kind = 'count', 
           height = 8, order = stateCount['customer_state'])

plt.show()

#### Product Category

In [7]:
catCount = pd.DataFrame(Data['product_category_name_english'].value_counts())
catCount = catCount.reset_index()
catCount = catCount.rename(columns = {'index': 'product_category', 'product_category_name_english':'product_category_count'})
catCount

product_category  product_category_count
0              bed_bath_table                   11272
1               health_beauty                    9728
2              sports_leisure                    8701
3             furniture_decor                    8416
4       computers_accessories                    7895
..                        ...                     ...
66      arts_and_craftmanship                      24
67                 la_cuisine                      14
68          cds_dvds_musicals                      14
69  fashion_childrens_clothes                       8
70      security_and_services                       2

[71 rows x 2 columns]

In [8]:
sb.catplot(y = 'product_category_name_english', data = Data, kind = 'count', 
           height = 14, order = catCount['product_category'])
plt.show()

#### Product ID

In [9]:
prodCount = pd.DataFrame(Data['product_id'].value_counts()).reset_index()
prodCount = prodCount.rename(columns = {'index': 'product_id', 'product_id':'product_id_count'})
prodCount

product_id  product_id_count
0      aca2eb7d00ea1a7b8ebd4e68314663af               527
1      99a4788cb24856965c36a24e339b6058               491
2      422879e10f46682990de24d770e7f83d               487
3      389d119b48cf3043d311335e499d9c6b               392
4      368c6c730842d78016ad823897a372db               391
...                                 ...               ...
32323  640fcadaeb5f06f26d6b07d366cebd34                 1
32324  1269d60fc5efbcf03e7eea49213091c6                 1
32325  57c406b73a01242b1d0bd1bcae6d7cc8                 1
32326  13cde3217a6ea7e5eb08bc8972ba9049                 1
32327  b0619e7ec34ce21283851c82f5189caa                 1

[32328 rows x 2 columns]

#### Data for Each State 

In [10]:
topCat_state = {}
topProd_state = {}

for state in stateCount['customer_state']: 
    
    DF = Data.loc[Data['customer_state'] == state]

    DF_cat = pd.DataFrame(DF['product_category_name_english'].value_counts())
    DF_cat = DF_cat.reset_index()
    DF_cat = DF_cat.rename(columns = {'index': 'product_category', 
                                      'product_category_name_english':'product_category_count'})
    
    DF_product = pd.DataFrame(DF['product_id'].value_counts())
    DF_product =  DF_product.reset_index().rename(columns = {'index': 'product_id', 
                                                            'product_id':'product_id_count'})
    
    topProd_state[state] = DF_product['product_id'][0]
    topCat_state[state] = DF_cat['product_category'][0]
    

recCat_state = pd.DataFrame(list(topCat_state.items()),columns = ['State','Product_Category_Recommendation'])
recProd_state = pd.DataFrame(list(topProd_state.items()),columns = ['State','Product_ID_Recommendation'])
    

#### Recommendation


In [11]:
Recomm_state = recCat_state.merge(recProd_state, on = 'State')

Recomm_state

State Product_Category_Recommendation         Product_ID_Recommendation
0     SP                  bed_bath_table  aca2eb7d00ea1a7b8ebd4e68314663af
1     RJ                  bed_bath_table  d1c427060a0f73f6b889a5c7c61f2ac4
2     MG                  bed_bath_table  d1c427060a0f73f6b889a5c7c61f2ac4
3     RS                  bed_bath_table  422879e10f46682990de24d770e7f83d
4     PR                 furniture_decor  a62e25e09e05e6faf31d90c6ec1aa3d1
5     SC                  sports_leisure  368c6c730842d78016ad823897a372db
6     BA                   health_beauty  3dd2a17168ec895c781a9191c1e95ad7
7     DF                   health_beauty  4c2394abfbac7ff59ec7a420918562fa
8     GO                  bed_bath_table  9571759451b1d780ee7c15012ea109d4
9     ES                  bed_bath_table  422879e10f46682990de24d770e7f83d
10    PE                   health_beauty  6cdd53843498f92890544667809f1595
11    CE                   health_beauty  bb50f2e236e5eea0100680137654686c
12    PA                   health_beauty  bb50f2e236e5eea0100680137654686c
13    MT                   health_beauty  422879e10f46682990de24d770e7f83d
14    MS                  sports_leisure  154e7e31ebfa092203795c972e5804a6
15    MA                   health_beauty  389d119b48cf3043d311335e499d9c6b
16    PB                   health_beauty  bb50f2e236e5eea0100680137654686c
17    PI                   health_beauty  3dd2a17168ec895c781a9191c1e95ad7
18    RN                   health_beauty  e5f3a09149ee7db697907f61e7366267
19    AL                   health_beauty  6cdd53843498f92890544667809f1595
20    SE                   health_beauty  6cdd53843498f92890544667809f1595
21    TO                   health_beauty  7c1ac37ea884ec9c46ebf155ffc53d50
22    RO                   health_beauty  c23381297fa734ea441a15d723199fb5
23    AM                   health_beauty  9226bd32a68934187cdb36243076ba99
24    AC                 furniture_decor  b81a05d0dd312ece2140846909f5ef81
25    AP                   health_beauty  68d9c8a95d2b7cb57efe9e1e929bdd6a
26    RR                  sports_leisure  1fe8e6e01596885617fa1c90d29c2f81

## Recommender System


### Collaborative Filtering 

Enables filtering of products based on historical preference of the customers.


### Matrix Factoristion

https://beckernick.github.io/matrix-factorization-recommender/

A matrix is broken down into a product of multiple matrices to predict scores for other products. It is relatively easier to implement with the limited data that we have (since not all products were rated by everyone). 


#### Data

In [12]:
mat_cat_df = pd.DataFrame(joinData[['customer_state', 'product_category_name_english', 'review_score']]).dropna().drop_duplicates()
mat_cat_df = mat_cat_df.rename(columns = {'product_category_name_english' : 'product_category'})

mat_cat_df = mat_cat_df.merge(catCount, on='product_category')
mat_cat_df = mat_cat_df.loc[mat_cat_df['product_category_count'] >100] #remove categorey with little data
mat_cat_df = mat_cat_df.drop(columns = ['product_category_count'])

mat_cat_df

customer_state  product_category  review_score
0                SP  office_furniture             4
1                SP  office_furniture             5
2                MA  office_furniture             1
3                RS  office_furniture             4
4                MG  office_furniture             5
...             ...               ...           ...
4140             PA   books_technical             4
4141             RJ   books_technical             4
4142             MG   books_technical             3
4143             CE   books_technical             3
4144             PA   books_technical             5

[4010 rows x 3 columns]

#### Score Matrix

In [13]:
mat_cat_pt = pd.pivot_table(data = mat_cat_df, index = 'customer_state', columns ='product_category', values = 'review_score').fillna(0)       
mat_cat_2D = mat_cat_pt.to_numpy()

mat_cat_pt

product_category  agro_industry_and_commerce  air_conditioning       art  \
customer_state                                                             
AC                                  0.000000          0.000000  0.000000   
AL                                  1.000000          5.000000  0.000000   
AM                                  0.000000          5.000000  5.000000   
AP                                  5.000000          0.000000  4.000000   
BA                                  3.666667          3.333333  3.000000   
CE                                  5.000000          3.000000  0.000000   
DF                                  0.000000          4.500000  3.333333   
ES                                  3.000000          2.500000  5.000000   
GO                                  3.250000          3.500000  4.000000   
MA                                  1.000000          5.000000  0.000000   
MG                                  3.250000          3.250000  4.500000   
MS                                  4.000000          0.000000  3.000000   
MT                                  4.500000          3.333333  5.000000   
PA                                  4.500000          0.000000  4.000000   
PB                                  2.500000          5.000000  1.000000   
PE                                  3.666667          1.000000  5.000000   
PI                                  4.000000          4.000000  4.000000   
PR                                  3.250000          3.250000  3.333333   
RJ                                  3.000000          3.000000  3.250000   
RN                                  0.000000          0.000000  3.000000   
RO                                  5.000000          0.000000  4.500000   
RR                                  0.000000          0.000000  0.000000   
RS                                  4.500000          3.333333  3.250000   
SC                                  4.500000          5.000000  4.000000   
SE                                  0.000000          0.000000  0.000000   
SP                                  3.000000          3.000000  3.000000   
TO                                  0.000000          0.000000  0.000000   

product_category     audio      auto      baby  bed_bath_table  \
customer_state                                                   
AC                0.000000  3.333333  5.000000        3.000000   
AL                3.000000  3.250000  4.000000        3.000000   
AM                5.000000  3.500000  4.000000        3.666667   
AP                0.000000  3.500000  0.000000        4.500000   
BA                3.000000  3.000000  3.000000        3.000000   
CE                5.000000  3.000000  3.000000        3.000000   
DF                3.000000  3.000000  3.000000        3.000000   
ES                3.000000  3.000000  3.250000        3.000000   
GO                3.250000  3.000000  3.000000        3.000000   
MA                4.500000  3.250000  3.333333        3.000000   
MG                3.000000  3.000000  3.000000        3.000000   
MS                5.000000  3.000000  3.000000        3.000000   
MT                2.666667  3.250000  3.000000        3.000000   
PA                1.000000  3.000000  3.000000        3.250000   
PB                2.000000  3.000000  3.000000        3.000000   
PE                4.500000  3.000000  3.000000        3.000000   
PI                4.000000  3.000000  3.500000        3.000000   
PR                3.000000  3.000000  3.000000        3.000000   
RJ                3.000000  3.000000  3.000000        3.000000   
RN                5.000000  3.000000  3.500000        3.000000   
RO                0.000000  3.500000  3.333333        2.750000   
RR                0.000000  5.000000  1.500000        3.000000   
RS                3.250000  3.000000  3.000000        3.000000   
SC                2.750000  3.000000  3.000000        3.000000   
SE                0.000000  3.250000  3.333333        3.000000   
SP           

#### Noramalise Data

In [14]:
from scipy.sparse.linalg import svds

In [15]:
score_mean_cat = np.mean(mat_cat_2D, axis = 1).reshape(-1, 1)
mat_cat_demeaned = mat_cat_2D - score_mean_cat


In [16]:
U, sigma, Vt = svds(mat_cat_demeaned, k = 26)
sigma = np.diag(sigma)

#### Predicted Scores from Decompsed Matrix

In [17]:
customer_predicted_score = np.dot(np.dot(U, sigma), Vt) + score_mean_cat
preds_df = pd.DataFrame(customer_predicted_score, columns = mat_cat_pt.columns, index = mat_cat_pt.index)
preds_df = preds_df.round(2)

preds_df

product_category  agro_industry_and_commerce  air_conditioning   art  audio  \
customer_state                                                                
AC                                     -0.00              0.00 -0.00  -0.00   
AL                                      1.00              5.00 -0.00   3.00   
AM                                      0.00              5.00  5.00   5.00   
AP                                      5.00              0.00  4.00  -0.00   
BA                                      3.67              3.33  3.00   3.00   
CE                                      5.00              3.00 -0.00   5.00   
DF                                     -0.00              4.50  3.33   3.00   
ES                                      3.00              2.50  5.00   3.00   
GO                                      3.25              3.50  4.00   3.25   
MA                                      1.00              5.00 -0.00   4.50   
MG                                      3.25              3.25  4.50   3.00   
MS                                      4.00              0.00  3.00   5.00   
MT                                      4.50              3.33  5.00   2.67   
PA                                      4.50              0.00  4.00   1.00   
PB                                      2.50              5.00  1.00   2.00   
PE                                      3.67              1.00  5.00   4.50   
PI                                      4.00              4.00  4.00   4.00   
PR                                      3.25              3.25  3.33   3.00   
RJ                                      3.00              3.00  3.25   3.00   
RN                                     -0.00              0.00  3.00   5.00   
RO                                      5.00             -0.00  4.50   0.00   
RR                                     -0.00              0.00 -0.00  -0.00   
RS                                      4.50              3.33  3.25   3.25   
SC                                      4.50              5.00  4.00   2.75   
SE                                     -0.00              0.00 -0.00  -0.00   
SP                                      3.01              2.99  3.00   3.01   
TO                                      0.00             -0.00  0.00   1.00   

product_category  auto  baby  bed_bath_table  books_general_interest  \
customer_state                                                         
AC                3.33  5.00            3.00                    4.00   
AL                3.25  4.00            3.00                    1.00   
AM                3.50  4.00            3.67                    5.00   
AP                3.50 -0.00            4.50                   -0.00   
BA                3.00  3.00            3.00                    3.33   
CE                3.00  3.00            3.00                    4.50   
DF                3.00  3.00            3.00                    3.50   
ES                3.00  3.25            3.00                    4.50   
GO                3.00  3.00            3.00                    3.00   
MA                3.25  3.33            3.00                    4.50   
MG                3.00  3.00            3.00                    3.25   
MS                3.00  3.00            3.00                    3.50   
MT                3.25  3.00            3.00                    4.00   
PA                3.00  3.00            3.25                    4.50   
PB                3.00  3.00            3.00                    4.50   
PE                3.00  3.00            3.00                    4.50   
PI                3.00  3.50            3.00                    5.00   
PR                3.00  3.00            3.00                    3.25   
RJ                3.00  3.00            3.00                    3.33   
RN                3.00  3.50            3.00                    4.50   
RO                3.50  3.33            2.75                    5.00   
RR                5.00  1.50            3.00        

#### Recommendation by Highest Predicted Score

In [18]:
def recommend (num, cus_state, preds_df):
    
    state_preds2 = pd.DataFrame(columns = ['customer_state','product_category', 'top_predicted_score'])
    
    for i in range (0,len(preds_df)):
        state_preds = pd.DataFrame(preds_df.iloc[[i]].transpose().to_records())
        state = state_preds.columns[1]  
        if cus_state == state:
            state_preds2 = state_preds.sort_values(by = state, ascending = False).head(num).rename(columns = {state:'top_predicted_score'})
            state_preds2['customer_state'] = state
    return state_preds2

In [25]:
cus_state = input('Enter state: ' )
num = input('Enter number of reccommendation: ')
#recommend (5, 'SP', preds_df)
recommend (int(num), cus_state, preds_df)

Enter state: AC
Enter number of reccommendation: 5


product_category  top_predicted_score customer_state
12         consoles_games                  5.0             AC
30  furniture_living_room                  5.0             AC
20            electronics                  5.0             AC
5                    baby                  5.0             AC
23          fashion_shoes                  5.0             AC

#### Recommendation by Highest Predicted Score for Each State

In [20]:
state_rec_MF = pd.DataFrame(columns = ['customer_state','product_category', 'top_predicted_score'])

for i in range (0,27):
    state_preds = pd.DataFrame(preds_df.iloc[[i]].transpose().to_records())
    state = state_preds.columns[1]  
    state_preds = state_preds.sort_values(by = state, ascending = False).head(1).rename(columns = {state:'top_predicted_score'})
    state_preds['customer_state'] = state
    state_rec_MF = state_rec_MF.append(state_preds)


In [21]:
state_rec_MF = state_rec_MF.set_index('customer_state')
state_rec_MF = state_rec_MF.sort_values(by = 'top_predicted_score', ascending = False)
state_rec_MF

product_category  top_predicted_score
customer_state                                                      
AC                               consoles_games                 5.00
MT                                          art                 5.00
SE                                         food                 5.00
SC                             air_conditioning                 5.00
RR                                watches_gifts                 5.00
RO                   agro_industry_and_commerce                 5.00
RN                                         food                 5.00
PI                                         food                 5.00
PE                                          art                 5.00
PB                                         food                 5.00
AL                                         food                 5.00
PA                                   food_drink                 5.00
MS                                         food                 5.00
MA              construction_tools_construction                 5.00
GO                            furniture_bedroom                 5.00
ES                                          art                 5.00
DF                       signaling_and_security                 5.00
CE                   agro_industry_and_commerce                 5.00
AP                   agro_industry_and_commerce                 5.00
AM                                          art                 5.00
TO               industry_commerce_and_business                 5.00
MG                                          art                 4.50
PR                                    computers                 4.50
BA                      costruction_tools_tools                 4.50
RS                   agro_industry_and_commerce                 4.50
RJ                           christmas_supplies                 3.33
SP                                   food_drink                 3.22